This Notebook will group stocks in sectors, find top n stocks of resptive sector based on user criteria, then will run a correlation, and find most divesrivfiable stocks (stocks with lowest correlation)

In [1]:
import time
import os 

start_time = time.time() # measure run time

WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')
if not os.getcwd().endswith('portfolio_py'):
    os.chdir(f'{WORKSPACE_DIR}/portfolio_py')
print(f'Current Working Directory: {os.getcwd()}')

from utils.finance_utils import *
from utils.helpers import separate_corr_pairs
from utils.config import TOP_N_STOCKS

Current Working Directory: /Users/blakeuribe/Desktop/portfolio_py

---------------------------------
finance_utils.py successfully loaded, updated last Feb. 24 2025 5:26
---------------------------------



---------------------------------
helpers.py successfully loaded, updated last Feb. 04 2025
---------------------------------


Updated on 03/05/2025 5:44


Sector Valuation: Group by sectors, and find best stock in each sector

In [2]:
filtered_valuation_df = pd.read_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/filtered_valuation_df.csv')

filtered_valuation_df

,marketCap,trailingPE,forwardPE,profitMargins,trailingEps,Tickers,Sharpe_ratios,sector
0,703305547776,12.742148,15.025687,0.35060,19.74,JPM,1.858474,Financial Services
1,299980750848,27.198757,24.624178,0.13930,9.66,TMUS,2.949385,Communication Services
2,254451417088,28.052631,16.400000,0.16958,2.28,CSCO,1.943530,Technology
3,339194085376,20.618420,14.288754,0.12316,6.84,BABA,1.902256,Consumer Cyclical
4,238111260672,25.523333,21.269445,0.18631,6.00,PM,3.073684,Consumer Defensive
5,161367425024,28.439419,23.433214,0.24778,172.91,BKNG,1.788805,Consumer Cyclical
6,217341214720,33.305920,38.645040,0.16940,6.08,GE,1.954915,Industrials
7,219176157184,9.932259,44.302160,0.39147,6.20,HSBC,3.175900,Financial Services
8,187924758528,17.570470,11.687500,0.08949,1.49,T,3.180367,Communication Services
9,154528923648,83.768000,37.530468,0.11071,1.25,BSX,2.878821,Healthcare


In [3]:
# possibly change group by  
print('\n----Filtering Df----')


top_stocks_on_profit = get_top_n_by_sector(filtered_valuation_df, filter_var='profitMargins', top_n=3)
top_stocks_on_sharpe = get_top_n_by_sector(filtered_valuation_df, filter_var='Sharpe_ratios', top_n=3)


top_stocks_on_profit


----Filtering Df----


,marketCap,trailingPE,forwardPE,profitMargins,trailingEps,Tickers,Sharpe_ratios,sector
0,69319065600,18.031248,21.988707,0.31877,17.28,APD,1.579210,Basic Materials
1,299980750848,27.198757,24.624178,0.13930,9.66,TMUS,2.949385,Communication Services
2,187924758528,17.570470,11.687500,0.08949,1.49,T,3.180367,Communication Services
3,161367425024,28.439419,23.433214,0.24778,172.91,BKNG,1.788805,Consumer Cyclical
4,62252224512,21.143510,16.186844,0.17452,10.94,RCL,2.422539,Consumer Cyclical
5,339194085376,20.618420,14.288754,0.12316,6.84,BABA,1.902256,Consumer Cyclical
6,94592425984,8.555046,10.457944,0.55097,6.54,MO,2.242306,Consumer Defensive
7,238111260672,25.523333,21.269445,0.18631,6.00,PM,3.073684,Consumer Defensive
8,88070160384,22.804598,8.442554,0.11861,1.74,BTI,2.283674,Consumer Defensive
9,54233931776,12.598575,12.027211,0.38784,4.21,MPLX,2.865751,Energy


Diversifaction: Find negative correlated stocks

In [ ]:
print('\n----Begining Diversifaction----')
top_profit_corr_pairs = get_corr_pairs_of_stocks(top_stocks_on_profit['Tickers'].tolist())

top_sharpe_corr_pairs = get_corr_pairs_of_stocks(top_stocks_on_sharpe['Tickers'].tolist())
top_profit_corr_pairs.head()


----Begining Diversifaction----


[*********************100%***********************]  22 of 22 completed
[*********************100%***********************]  22 of 22 completed


,Correlation
BABA_GDDY,0.504765
CQP_DUK,0.516764
BABA_SYF,0.518448
BABA_BKNG,0.603087
BABA_VST,0.613561
BABA_GRMN,0.614171
DUK_VST,0.615435
BKNG_DUK,0.621707
DUK_SYF,0.633461
BABA_CQP,0.643034


Seperate low corr pairs

In [5]:
corr_profit_pairs = top_profit_corr_pairs.index.to_list()
corr_sharpe_pairs = top_sharpe_corr_pairs.index.to_list()

mpt_stocks_for_sharpe = separate_corr_pairs(corr_sharpe_pairs, top_n=TOP_N_STOCKS)
mpt_stocks_for_profit = separate_corr_pairs(corr_profit_pairs, top_n=TOP_N_STOCKS)
print(f'Top Stocks based on Sharpe: {mpt_stocks_for_sharpe}')
print(f'Top Stocks based on Profit Margin: {mpt_stocks_for_profit}')

Top Stocks based on Sharpe: ['BABA' 'GDDY' 'CQP' 'DUK' 'K' 'BKNG' 'VST' 'GRMN']
Top Stocks based on Profit Margin: ['BABA' 'GDDY' 'CQP' 'DUK' 'SYF' 'BKNG' 'VST' 'GRMN']


In [6]:
with open('utils/config.py', 'a') as f:
    f.write('\n# New Entry from portfolio_selection.ipynb')
    f.write(f'\nmpt_stocks_for_sharpe = {(mpt_stocks_for_sharpe).tolist()}\n')
    f.write(f'mpt_stocks_for_profit = {(mpt_stocks_for_profit).tolist()}\n')

In [7]:
filtered_valuation_df[filtered_valuation_df['Tickers'].isin(mpt_stocks_for_sharpe)]

,marketCap,trailingPE,forwardPE,profitMargins,trailingEps,Tickers,Sharpe_ratios,sector
3,339194085376,20.618420,14.288754,0.12316,6.84,BABA,1.902256,Consumer Cyclical
5,161367425024,28.439419,23.433214,0.24778,172.91,BKNG,1.788805,Consumer Cyclical
15,89471606784,20.215790,18.175080,0.15113,5.70,DUK,2.023362,Utilities
23,43004362752,18.150215,18.386957,0.15438,6.99,VST,1.947318,Utilities
24,43254341632,30.753761,30.135387,0.22415,7.31,GRMN,1.872240,Technology
27,24921063424,27.291021,26.274218,0.20487,6.46,GDDY,2.000876,Technology
28,28435443712,21.229382,68.641660,0.10534,3.88,K,3.004948,Consumer Defensive
29,30708004864,14.927058,14.753488,0.28837,4.25,CQP,2.665342,Energy
